In [1]:
#from ib_insync import *
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels import regression,stats
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta
from collections import deque
from os import listdir
from os.path import isfile, join
from collections import defaultdict
from operator import itemgetter 

In [2]:
def get_ratio(country, adr, fx_dict):
    adr_path = f'eric_jh_data/{country}/{adr}/adr.csv'
    stock_path =  f'eric_jh_data/{country}/{adr}/underlying.csv'
    fx_path = fx_dict[country][0]
    fx_type =  fx_dict[country][1]
    
#     Old exception code that we probably don't need anymore
#     try:
#         adr_df = pd.read_csv(adr_path, index_col = 0).rename(columns = {'close':'adr_close', 'open':'adr_open'})
#     except:
#         print(f"no data for ADR data of {adr} from {country}")
#         return None
#     try:
#         stock_df = pd.read_csv(stock_path, index_col = 0).rename(columns = {'close':'stock_close', 'open':'stock_open'})
#     except:
#         print(f"no data for underlying data of {adr} from {country}")
#         return None

    adr_df = pd.read_csv(adr_path, index_col = 0).rename(columns = {'close':'adr_close', 'open':'adr_open'})
    stock_df = pd.read_csv(stock_path, index_col = 0).rename(columns = {'close':'stock_close', 'open':'stock_open'})
    
    fx_df = pd.read_csv(fx_path, index_col = 0)

    merged_df = pd.merge(adr_df.loc[:,['date', 'adr_open','adr_close']], stock_df.loc[:,['date', 'stock_open','stock_close']])
    merged_df = pd.merge(merged_df, fx_df)

    if fx_type == 1:
        merged_df['stock_open_usd'] = merged_df['stock_open']/((merged_df['avg_bid_non_us_at'] + merged_df['avg_ask_non_us_at'])/2)
    else:
        merged_df['stock_open_usd'] = merged_df['stock_open']*((merged_df['avg_bid_non_us_at'] + merged_df['avg_ask_non_us_at'])/2)
        

    merged_df["ratio"] = merged_df['stock_open_usd']/merged_df['adr_close']
    
    ratio_geq_1 = True
    
    if np.mean(merged_df["ratio"] < 1):
        merged_df["ratio"] = 1/merged_df["ratio"]
        ratio_geq_1 = False
    
    return ratio_geq_1, np.round(np.mean(merged_df["ratio"]), 4)

In [3]:
mypath = 'eric_jh_data/'
countries = ['Australia', 'Japan', 'China']
adr_dict = {}
fx_dict = {'Australia':('eric_jh_data/Forex/AUD_USD_new.csv',0),
           'Japan':('eric_jh_data/Forex/USD_JPY_new.csv',1),
           'China':('eric_jh_data/Forex/USD_HKD_new.csv',1)}

# Store ratios
for country in countries:
    countrypath = mypath + country
    adr_names =  [f for f in listdir(countrypath) if not isfile(join(countrypath, f))] #grab all adr names of the country
    adr_dict[country] = adr_names
    
    for adr in adr_names:
        ratio_geq_1, ratio = get_ratio(country, adr, fx_dict)
        if adr == "ACH_2600" or adr == "BGNE_6160":
            rounded_ratio = float('%.2g' % ratio)
        else:
            rounded_ratio = float('%.1g' % ratio)
        print("Country: {}, ADR_Stock: {}, Estimated Ratio (4 d.p.): {}, Implied Ratio (2 s.f.): {}".format(country, adr, ratio, rounded_ratio))
        ratio_df = pd.DataFrame({"ratio_geq_1" : [ratio_geq_1], "ratio" : [rounded_ratio]})
        ratio_df.to_csv(f'eric_jh_data/{country}/{adr}/ratio.csv')

Country: Australia, ADR_Stock: JHX_JHX, Estimated Ratio (4 d.p.): 1.0043, Implied Ratio (2 s.f.): 1.0
Country: Australia, ADR_Stock: PLL_PLL, Estimated Ratio (4 d.p.): 101.5788, Implied Ratio (2 s.f.): 100.0
Country: Australia, ADR_Stock: GENE_GTG, Estimated Ratio (4 d.p.): 595.5978, Implied Ratio (2 s.f.): 600.0
Country: Australia, ADR_Stock: ATHE_ATH, Estimated Ratio (4 d.p.): 59.4889, Implied Ratio (2 s.f.): 60.0
Country: Australia, ADR_Stock: IMMP_IMM, Estimated Ratio (4 d.p.): 9.9082, Implied Ratio (2 s.f.): 10.0
Country: Australia, ADR_Stock: WBK_WBC, Estimated Ratio (4 d.p.): 0.9997, Implied Ratio (2 s.f.): 1.0
Country: Australia, ADR_Stock: KZIA_KZA, Estimated Ratio (4 d.p.): 10.0346, Implied Ratio (2 s.f.): 10.0
Country: Australia, ADR_Stock: MESO_MSB, Estimated Ratio (4 d.p.): 5.0155, Implied Ratio (2 s.f.): 5.0
Country: Australia, ADR_Stock: IMRN_IMC, Estimated Ratio (4 d.p.): 39.4289, Implied Ratio (2 s.f.): 40.0
Country: Japan, ADR_Stock: HMC_7267, Estimated Ratio (4 d.p.)

In [4]:
def data_processing(country, adr, fx_dict):
    adr_path = f'eric_jh_data/{country}/{adr}/adr.csv'
    stock_path =  f'eric_jh_data/{country}/{adr}/underlying.csv'
    ratio_path = f'eric_jh_data/{country}/{adr}/ratio.csv'
    fx_path = fx_dict[country][0]
    fx_type =  fx_dict[country][1]
    
#     Old exception code that we probably don't need anymore
#     try:
#         adr_df = pd.read_csv(adr_path, index_col = 0).rename(columns = {'close':'adr_close', 'open':'adr_open'})
#     except:
#         print(f"no data for ADR data of {adr} from {country}")
#         return None
#     try:
#         stock_df = pd.read_csv(stock_path, index_col = 0).rename(columns = {'close':'stock_close', 'open':'stock_open'})
#     except:
#         print(f"no data for underlying data of {adr} from {country}")
#         return None

    adr_df = pd.read_csv(adr_path, index_col = 0).rename(columns = {'close':'adr_close', 'open':'adr_open'})
    stock_df = pd.read_csv(stock_path, index_col = 0).rename(columns = {'close':'stock_close', 'open':'stock_open'})
    fx_df = pd.read_csv(fx_path, index_col = 0)
    ratio_df = pd.read_csv(ratio_path, index_col = 0)

    # Invert fx data
    if fx_type == 0:
        inverted_fx_df = 1/fx_df.iloc[:,[2,1,4,3,6,5,8,7]].copy()
        inverted_fx_df.columns = fx_df.columns[1:]
        fx_df.iloc[:,1:] = inverted_fx_df
    merged_df = pd.merge(adr_df.loc[:,['date', 'adr_open','adr_close']], stock_df.loc[:,['date', 'stock_open','stock_close']])
    merged_df = pd.merge(merged_df, fx_df)
    ratio_geq_1, ratio = ratio_df["ratio_geq_1"].item(), ratio_df["ratio"].item()

#     If ratio >= 1, we shall buy 1 stock, and sell multiple adrs
#     If ratio < 1, we shall sell 1 adr, and buy multiple stocks
    if ratio_geq_1:
        merged_df["stock_num_per_unit"] = 1
        merged_df["adr_num_per_unit"] = ratio
        merged_df["stock_open_per_unit"] = merged_df["stock_open"]
        merged_df["stock_close_per_unit"] = merged_df["stock_close"]
        merged_df["adr_open_per_unit"] = merged_df["adr_open"]*ratio
        merged_df["adr_close_per_unit"] = merged_df["adr_close"]*ratio
    else:
        merged_df["stock_num_per_unit"] = ratio
        merged_df["adr_num_per_unit"] = 1
        merged_df["stock_open_per_unit"] = merged_df["stock_open"]*ratio
        merged_df["stock_close_per_unit"] = merged_df["stock_close"]*ratio
        merged_df["adr_open_per_unit"] = merged_df["adr_open"]
        merged_df["adr_close_per_unit"] = merged_df["adr_close"]
        
    return merged_df

In [5]:
for country in countries:
    countrypath = mypath + country
    adr_names =  [f for f in listdir(countrypath) if not isfile(join(countrypath, f))] #grab all adr names of the country
    adr_dict[country] = adr_names
    
    for adr in adr_names:
        merged_df = data_processing(country, adr, fx_dict)

In [6]:
# stock_num_per_unit is how many stocks we would buy for 1 "unit" of trade
# avg_bid_non_us_before is how much foreign currency we can buy with 1 USD, 1 minute before the Asian market opens
# avg_bid_non_us_at is how much foreign currency we can buy with 1 USD, when the Asian market opens
# avg_bid_us_before is how much foreign currency we can buy with 1 USD, 1 minute before the US market opens
# avg_bid_us_at is how much foreign currency we can buy with 1 USD, when the US market opens
# All dates are in local time: so in sequential order, it will go stock_open, stock_close, adr_open, adr_close
merged_df.tail()

,date,adr_open,adr_close,stock_open,stock_close,avg_bid_non_us_before,avg_ask_non_us_before,avg_bid_non_us_at,avg_ask_non_us_at,avg_bid_us_before,avg_ask_us_before,avg_bid_us_at,avg_ask_us_at,stock_num_per_unit,adr_num_per_unit,stock_open_per_unit,stock_close_per_unit,adr_open_per_unit,adr_close_per_unit
1436,2021-03-30,23.81,23.80,1.87,1.87,7.77416,7.77418,7.77422,7.77424,7.77568,7.77570,7.77564,7.77566,100.0,1,187.0,187.0,23.81,23.80
1437,2021-03-31,23.67,23.64,1.87,1.84,7.77471,7.77473,7.77460,7.77463,7.77412,7.77415,7.77412,7.77416,100.0,1,187.0,184.0,23.67,23.64
1438,2021-04-01,23.62,23.60,1.85,1.84,7.77372,7.77375,7.77372,7.77375,7.77682,7.77684,7.77682,7.77684,100.0,1,185.0,184.0,23.62,23.60
1439,2021-04-07,23.67,23.44,1.83,1.85,7.77681,7.77685,7.77680,7.77687,7.78626,7.78628,7.78627,7.78628,100.0,1,183.0,185.0,23.67,23.44
1440,2021-04-08,24.18,24.21,1.85,1.91,7.78459,7.78461,7.78461,7.78463,7.77724,7.77726,7.77724,7.77725,100.0,1,185.0,191.0,24.18,24.21


In [7]:
"""
Variant 1 - Begin each trade on Asian market open (Evaluate after US market closes)

To open a position, we check the CLOSE price of adr, compared it to CLOSE px of 
stock of the same row. We buy the stock on the next trading OPEN for Asian/US market

To close a position,  we check the CLOSE price of adr, compared it to CLOSE px of 
stock of the same row. We sell the stock on the next trading next OPEN for Asian/US market

For each row:
    stock_open, stock_close, adr_open, adr_close
    After these 4 events, assess condition (right before the Asian market opens ~ 6.59PM EST)
    Place trade on next row (First trade stock on Asian market open, then trade ADR on US market open)
    
start_date: First date (EST) we may place a trade
end_date: Last date (EST) we may place a trade
portfolio_values: Stores value of portfolio at each date from one day before the start_date, to the end_date, when Asian market opens (EST ~ 7:00 PM)
"""
def pairs_trade_v1(merged_df, lookback = 100, cash = 100000, entry = 1, exit = 0, 
                   stop_loss = 3, start_date = "2016-01-01", end_date = "2021-01-31", 
                   bps = 20):
    
    short_multiplier = 1 - 0.0001*bps
    long_multiplier = 1 + 0.0001*bps
    starting_cash = cash
    stock_pos, adr_pos = 0, 0
    prev_cash, prev_adr_pos = cash, adr_pos
    diff_record_bid = deque(maxlen = lookback)
    diff_record_ask = deque(maxlen = lookback)
    trade_records = []
    portfolio_values = []
    dates = []
    hits = []

    for index, row in merged_df.iterrows():

        if index+1 < len(merged_df) and index > 0:
            
            # Add portfolio value for the day before
            prev_date = merged_df.loc[index - 1, "date"]
            if row["date"] >= start_date and prev_date <= end_date:
                dates.append(prev_date)
                portfolio_values.append(prev_cash + prev_adr_pos*merged_df.loc[index - 1, 'adr_close'] + stock_pos*(row['stock_open']/row['avg_ask_non_us_at']))

            diff_record_bid.append(row['adr_close_per_unit'] - row['stock_close_per_unit']/merged_df.loc[index+1,'avg_bid_non_us_before'])
            diff_record_ask.append(row['adr_close_per_unit'] - row['stock_close_per_unit']/merged_df.loc[index+1,'avg_ask_non_us_before'])

            # We place one trade the day itself (Asian), one trade the day after (US)
            if len(diff_record_bid) < lookback or row["date"] < start_date or merged_df.loc[index+1, "date"] > end_date:
                continue
            
            prev_cash, prev_adr_pos = cash, adr_pos

            # If we have passed the initial lookback window and are in the specified dates
            # enter the position if diff is significant
            ask_mean = np.array(diff_record_ask).mean()
            bid_std = np.array(diff_record_bid).std()
            ask_std = np.array(diff_record_ask).std()
            
            if diff_record_bid[-1] > ask_mean + entry*bid_std and diff_record_bid[-1] <= ask_mean + stop_loss*bid_std:
                if stock_pos == 0 and adr_pos == 0:
                    portfolio_value_before_entering = cash
                    units = min(int((0.5*cash)/row['adr_close_per_unit']),int((0.5*cash)/(row['stock_close_per_unit']/merged_df.loc[index+1,'avg_bid_non_us_before'])))
                    adr_quantity = int(units*row["adr_num_per_unit"])
                    stock_quantity = int(units*row["stock_num_per_unit"])
                    
                    stock_pos += stock_quantity
                    stock_px_fx = merged_df.loc[index+1,'stock_open']*long_multiplier
                    stock_px = stock_px_fx/merged_df.loc[index+1,'avg_bid_non_us_at']
                    cash -= stock_px*stock_quantity
                    prev_cash, prev_adr_pos = cash, adr_pos
                    
                    adr_pos -= adr_quantity
                    adr_px = merged_df.loc[index+1,'adr_open']*short_multiplier
                    cash += adr_quantity*adr_px

                    trade_records.append("Opening positions:\n")
                    # Times in EST
                    trade_records.append(f"We bought {stock_quantity} shares of underlying stock at the price of {stock_px} USD ({stock_px_fx} foreign dollars) on {row['date']}\n")
                    trade_records.append(f"We sold {adr_quantity} shares of ADR at the price of {adr_px} on {merged_df.loc[index+1,'date']}\n")

            # When do we exit the position?
            elif diff_record_ask[-1] < ask_mean + exit*ask_std or diff_record_bid[-1] > ask_mean + stop_loss*bid_std:
                if stock_pos > 0 and adr_pos < 0 : # If we have positions in the stocks, we liquidate the position
                    stock_px_fx = merged_df.loc[index+1,'stock_open']*short_multiplier
                    stock_px = stock_px_fx/merged_df.loc[index+1,'avg_ask_non_us_at']
                    cash += stock_pos*stock_px
                    prev_cash = cash
                    
                    adr_px = merged_df.loc[index+1,'adr_open']*long_multiplier
                    cash -= abs(adr_pos)*adr_px
                    trade_records.append("Closing positions:\n")
                    trade_records.append(f"We sold {stock_pos} shares of underlying stock at the price of {stock_px} USD ({stock_px_fx} foreign dollars) on {row['date']}\n")
                    trade_records.append(f"We bought {-adr_pos} shares of ADR at the price of {adr_px} on {merged_df.loc[index+1,'date']}\n")
                    stock_pos, adr_pos = 0, 0
                    if cash > portfolio_value_before_entering:
                        hits.append(1)
                    else:
                        hits.append(0)

    ret = (portfolio_values[-1] - starting_cash)/starting_cash
    
    return ret, trade_records, portfolio_values, hits, dates

In [8]:
def calc_max_drawdown(portfolio_values):
    peak, trough = portfolio_values[0], portfolio_values[0]
    max_drawdown = 0
    for i in range(1, len(portfolio_values)):
        if portfolio_values[i] < trough:
            trough = portfolio_values[i]
            max_drawdown = max(max_drawdown, (peak - trough)/peak)
        elif portfolio_values[i] > peak:
            peak, trough = portfolio_values[i], portfolio_values[i]
    return max_drawdown

In [9]:
def report_and_store_statistics(pairs_trade_strategy, filename):
    for country in countries:
        countrypath = mypath + country
        adr_names =  [f for f in listdir(countrypath) if not isfile(join(countrypath, f))] #grab all adr names of the country
        adr_dict[country] = adr_names

        for adr in adr_names:
            merged_df = data_processing(country, adr, fx_dict)
            ret, trade_records, portfolio_values, hits, dates = pairs_trade_strategy(merged_df)
            ret = np.round(ret*100, 2)
            hit_ratio = None
            logs = [f'The return of ADR_underlying pairs trading for {adr} from {country} is {0.00}%, no trades were placed.\n']
            if hits:
                hit_ratio = np.round(np.mean(hits)*100,2)
                max_drawdown = np.round(calc_max_drawdown(portfolio_values)*100,2)
                logs = [f'The return of ADR_underlying pairs trading for {adr} from {country} is {ret}%\nThe hit ratio is {hit_ratio}%\nThe max drawdown is {max_drawdown}%\n']
                print("Country: {}, ADR_Stock: {}, Return: {}%, Hit Ratio: {}%, Max Drawdown: {}%".format(country, adr, ret, hit_ratio, max_drawdown))
            else:
                print("Country: {}, ADR_Stock: {}, Return: {}%, Hit Ratio: None, Max Drawdown: 0.00%".format(country, adr, ret))
            logs = logs + trade_records 
            fname = f'eric_jh_data/{country}/{adr}/' + filename
            f = open(fname, 'w')
            f.writelines(logs)
            f.close()
    return dates, portfolio_values

In [ ]:
last_dates, last_portfolio_values = report_and_store_statistics(pairs_trade_v1, 'logs1.txt')

Country: Australia, ADR_Stock: JHX_JHX, Return: -40.04%, Hit Ratio: 33.66%, Max Drawdown: 43.12%
Country: Australia, ADR_Stock: PLL_PLL, Return: 99.12%, Hit Ratio: 67.86%, Max Drawdown: 46.62%
Country: Australia, ADR_Stock: GENE_GTG, Return: 425.77%, Hit Ratio: 79.63%, Max Drawdown: 16.28%
Country: Australia, ADR_Stock: ATHE_ATH, Return: 451.07%, Hit Ratio: 84.44%, Max Drawdown: 18.97%
Country: Australia, ADR_Stock: IMMP_IMM, Return: 195.11%, Hit Ratio: 75.0%, Max Drawdown: 20.91%
Country: Australia, ADR_Stock: WBK_WBC, Return: -17.92%, Hit Ratio: 36.56%, Max Drawdown: 22.31%
Country: Australia, ADR_Stock: KZIA_KZA, Return: 40.82%, Hit Ratio: 73.97%, Max Drawdown: 66.27%
Country: Australia, ADR_Stock: MESO_MSB, Return: 50.74%, Hit Ratio: 68.66%, Max Drawdown: 47.71%
Country: Australia, ADR_Stock: IMRN_IMC, Return: 365.74%, Hit Ratio: 80.49%, Max Drawdown: 8.39%
Country: Japan, ADR_Stock: HMC_7267, Return: -31.44%, Hit Ratio: 35.71%, Max Drawdown: 35.41%
Country: Japan, ADR_Stock: CAJ_7

In [ ]:
def plot_returns(dates, portfolio_values, num_xticks = 5):
    plt.plot(portfolio_values)
    xticks_indices = np.arange(0, len(dates), (len(dates)-1)// num_xticks)
    plt.xticks(xticks_indices, itemgetter(*xticks_indices)(dates), rotation = 45)
    plt.show();

In [ ]:
plot_returns(last_dates, last_portfolio_values)

In [ ]:
"""
Variant 2 - Begin each trade on US market open (Evaluate after Asian market closes)

To open a position, we check the CLOSE price of adr of the previous row, compared to CLOSE px of 
stock of the current row. We buy the stock on the next trading OPEN for Asian/US market

To close a position, we check the CLOSE price of adr of the previous row, compared to CLOSE px of 
stock of the current row. We sell the stock on the next trading next OPEN for Asian/US market

For each row:
    stock_open, stock_close, (assess), adr_open, adr_close
    After first 2 events events, assess condition (right before the US market opens ~ 9.29AM EST)
    Place trade on current and next row (First trade ADR on US market open, then trade stock on Asian market open)
    
start_date: First date (EST) we may place a trade
end_date: Last date (EST) we may place a trade
portfolio_values: Stores value of portfolio at each date from one day before the start_date, to the end_date, when the Asian market opens
"""
def pairs_trade_v2(merged_df, lookback = 100, cash = 100000, entry = 1, exit = 0, 
                   stop_loss = 3, start_date = "2016-01-01", end_date = "2021-01-31", 
                   bps = 20):

    short_multiplier = 1 - 0.0001*bps
    long_multiplier = 1 + 0.0001*bps
    starting_cash = cash
    stock_pos, adr_pos = 0, 0
    diff_record_bid = deque(maxlen = lookback)
    diff_record_ask = deque(maxlen = lookback)
    trade_records = []
    portfolio_values = []
    dates = []
    hits = []

    for index, row in merged_df.iterrows():
        
        if index+1 < len(merged_df) and index > 0:
            
            # Add portfolio value for the day before
            prev_date = merged_df.loc[index - 1, "date"]
            if row["date"] >= start_date and prev_date <= end_date:
                dates.append(prev_date)
                portfolio_values.append(cash + adr_pos*merged_df.loc[index - 1, 'adr_close'] + stock_pos*(row['stock_open']/row['avg_ask_non_us_at']))

            # check if there is a px diff between close and stock_close effective
            diff_record_bid.append(merged_df.loc[index-1,'adr_close_per_unit'] - row['stock_close_per_unit']/row['avg_bid_us_before'])
            diff_record_ask.append(merged_df.loc[index-1,'adr_close_per_unit'] - row['stock_close_per_unit']/row['avg_ask_us_before'])

            # We place both trades the day itself
            # If len(diff_record) < lookback, we do not place any trade
            if len(diff_record_bid) < lookback or row["date"] < start_date or row["date"] > end_date:
                continue

            # If we have passed the initial lookback window and are in the specified dates
            # enter the position if diff is significant
            ask_mean = np.array(diff_record_ask).mean()
            bid_std = np.array(diff_record_bid).std()
            ask_std = np.array(diff_record_ask).std()
            
            if diff_record_bid[-1] > ask_mean + entry*bid_std and diff_record_bid[-1] <= ask_mean + stop_loss*bid_std:
                if stock_pos == 0 and adr_pos == 0:
                    portfolio_value_before_entering = cash
                    units = min(int((0.5*cash)/merged_df.loc[index-1,'adr_close_per_unit']),int((0.5*cash)/(row['stock_close_per_unit']/row['avg_bid_us_before'])))
                    adr_quantity = int(units*row["adr_num_per_unit"])
                    stock_quantity = int(units*row["stock_num_per_unit"])
                    adr_pos -= adr_quantity
                    adr_px = row['adr_open']*short_multiplier
                    cash += adr_quantity*adr_px

                    stock_pos += stock_quantity
                    stock_px_fx = merged_df.loc[index+1,'stock_open']*long_multiplier
                    stock_px = stock_px_fx/merged_df.loc[index+1,'avg_bid_non_us_at']
                    # The actual px we get to trade is on the next day for asian market
                    cash -= stock_px*stock_quantity
                    trade_records.append("Opening positions:\n")
                    # Times in EST
                    trade_records.append(f"We sold {adr_quantity} shares of ADR at the price of {adr_px} on {row['date']}\n")
                    trade_records.append(f"We bought {stock_quantity} shares of underlying stock at the price of {stock_px} USD ({stock_px_fx} foreign dollars) on {row['date']}\n")

            # When do we exit the position?
            elif diff_record_ask[-1] < ask_mean + exit*ask_std or diff_record_bid[-1] > ask_mean + stop_loss*bid_std:
                if stock_pos > 0 and adr_pos < 0 : # If we have positions in the stocks, we liquidate the position
                    adr_px = row['adr_open']*long_multiplier
                    cash -= abs(adr_pos)*adr_px
                    stock_px_fx = merged_df.loc[index+1,'stock_open']*short_multiplier
                    stock_px = stock_px_fx/merged_df.loc[index+1,'avg_ask_non_us_at']
                    cash += stock_pos*stock_px
                    trade_records.append("Closing positions:\n")
                    trade_records.append(f"We bought {-adr_pos} shares of ADR at the price of {adr_px} on {row['date']}\n")
                    trade_records.append(f"We sold {stock_pos} shares of underlying stock at the price of {stock_px} USD ({stock_px_fx} foreign dollars) on {row['date']}\n")
                    stock_pos, adr_pos = 0, 0
                    if cash > portfolio_value_before_entering:
                        hits.append(1)
                    else:
                        hits.append(0)

    ret = (portfolio_values[-1] - starting_cash)/starting_cash
    
    return ret, trade_records, portfolio_values, hits, dates

In [ ]:
last_dates, last_portfolio_values = report_and_store_statistics(pairs_trade_v2, 'logs2.txt')

In [ ]:
plot_returns(last_dates, last_portfolio_values)

In [ ]:
"""
Variant 3a 
- Begin each trade on either US market open or Asian market open
- Regressions are done for the similar "type" of trade
    i.e. if we are entering at a certain time, we do a regression based on the values obtained at the same time each day

For each row:
    stock_open, stock_close, (assess condition 1), adr_open, adr_close, (assess condition 2)
    If not condition 2 - No action taken: 
        After first 2 events, Assess condition 1 (right before the US market opens ~ 9.29AM EST)
        If condition 1:
            Place trade on current and next row (First trade ADR on US market open, then trade stock on Asian market open)
    If not condition 1 - No action taken:
        After next 2 events occur, assess condition 2
        If condition 2:
            Place trade on next row (First trade ADR on Asian market open, then trade stock on US market open)
    
start_date: First date (EST) we may place a trade
end_date: Last date (EST) we may place a trade
portfolio_values: Stores value of portfolio at each date from one day before the start_date, to the end_date, when the Asian market opens
"""
def pairs_trade_v3a(merged_df, lookback = 100, cash = 100000, entry_cond1_val = 1, entry_cond2_val = 1, 
                    exit_cond1_val = 0, exit_cond2_val = 0, stop_loss_cond1 = 3, stop_loss_cond2 = 3, 
                    start_date = "2016-01-01", end_date = "2021-01-31", bps = 20):

    short_multiplier = 1 - 0.0001*bps
    long_multiplier = 1 + 0.0001*bps
    starting_cash = cash
    stock_pos, adr_pos = 0, 0
    prev_cash, prev_adr_pos = cash, adr_pos
    diff_record_bid_cond1 = deque(maxlen = lookback)
    diff_record_ask_cond1 = deque(maxlen = lookback)
    diff_record_bid_cond2 = deque(maxlen = lookback)
    diff_record_ask_cond2 = deque(maxlen = lookback)
    trade_records = []
    portfolio_values = []
    dates = []
    hits = []
    enter_cond1, exit_cond1, enter_cond2, exit_cond2 = False, False, False, False

    for index, row in merged_df.iterrows():
                    
        # check if there is a px diff between close and stock_close effective
        # If len(diff_record) < lookback, we do not place any trade
        if index+1 < len(merged_df) and index > 0:
            
            # Add portfolio value for the day before
            prev_date = merged_df.loc[index - 1, "date"]
            if row["date"] >= start_date and prev_date <= end_date:
                dates.append(prev_date)
                portfolio_values.append(prev_cash + prev_adr_pos*merged_df.loc[index - 1, 'adr_close'] + stock_pos*(row['stock_open']/row['avg_ask_non_us_at']))
            
            # Before US Market Opens
            diff_record_bid_cond1.append(merged_df.loc[index-1,'adr_close_per_unit'] - row['stock_close_per_unit']/row['avg_bid_us_before'])
            diff_record_ask_cond1.append(merged_df.loc[index-1,'adr_close_per_unit'] - row['stock_close_per_unit']/row['avg_ask_us_before'])
            # Before Asian Market Opens
            diff_record_bid_cond2.append(row['adr_close_per_unit'] - row['stock_close_per_unit']/merged_df.loc[index+1,'avg_bid_non_us_before'])
            diff_record_ask_cond2.append(row['adr_close_per_unit'] - row['stock_close_per_unit']/merged_df.loc[index+1,'avg_ask_non_us_before'])


            # row["date"] is between start_date (inclusive) and end_date (inclusive)
            if len(diff_record_bid_cond1) < lookback or row["date"] < start_date or row["date"] > end_date:
                continue

            prev_cash, prev_adr_pos = cash, adr_pos
                
            # If we have passed the initial lookback window and are in the specified dates
            # enter the position if diff is significant
            ask_mean_cond1 = np.array(diff_record_ask_cond1).mean()
            bid_std_cond1 = np.array(diff_record_bid_cond1).std()
            ask_std_cond1 = np.array(diff_record_ask_cond1).std()
            ask_mean_cond2 = np.array(diff_record_ask_cond2).mean()
            bid_std_cond2 = np.array(diff_record_bid_cond2).std()
            ask_std_cond2 = np.array(diff_record_ask_cond2).std()
            
            if not (enter_cond2 or exit_cond2):
                # Check and possibly trade condition 1
                enter_cond1 = (diff_record_bid_cond1[-1] > ask_mean_cond1 + entry_cond1_val*bid_std_cond1 
                               and diff_record_bid_cond1[-1] <= ask_mean_cond1 + stop_loss_cond1*bid_std_cond1
                               and stock_pos == 0 and adr_pos == 0)
                exit_cond1 = ((diff_record_ask_cond1[-1] < ask_mean_cond1 + exit_cond1_val*ask_std_cond1 
                              or diff_record_bid_cond1[-1] > ask_mean_cond1 + stop_loss_cond1*bid_std_cond1)
                              and stock_pos > 0 and adr_pos < 0)
                    
                if enter_cond1:
                    portfolio_value_before_entering = cash
                    units = min(int((0.5*cash)/merged_df.loc[index-1,'adr_close_per_unit']),int((0.5*cash)/(row['stock_close_per_unit']/row['avg_bid_us_before'])))
                    adr_quantity = int(units*row["adr_num_per_unit"])*short_multiplier
                    stock_quantity = int(units*row["stock_num_per_unit"])
                    adr_pos -= adr_quantity
                    adr_px = row['adr_open']
                    cash += adr_quantity*adr_px

                    stock_pos += stock_quantity
                    stock_px_fx = merged_df.loc[index+1,'stock_open']*long_multiplier
                    stock_px = stock_px_fx/merged_df.loc[index+1,'avg_bid_non_us_at']
                    # The actual px we get to trade is on the next day for asian market
                    cash -= stock_px*stock_quantity
                    prev_cash, prev_adr_pos = cash, adr_pos
                    trade_records.append("Opening positions:\n")
                    # Times in EST
                    trade_records.append(f"We sold {adr_quantity} shares of ADR at the price of {adr_px} on {row['date']}\n")
                    trade_records.append(f"We bought {stock_quantity} shares of underlying stock at the price of {stock_px} USD ({stock_px_fx} foreign dollars) on {row['date']}\n")
                    
                elif exit_cond1:
                    
                    adr_px = row['adr_open']*long_multiplier
                    cash -= abs(adr_pos)*adr_px
                    stock_px_fx = merged_df.loc[index+1,'stock_open']*short_multiplier
                    stock_px = stock_px_fx/merged_df.loc[index+1,'avg_ask_non_us_at']
                    cash += stock_pos*stock_px
                    trade_records.append("Closing positions:\n")
                    trade_records.append(f"We bought {-adr_pos} shares of ADR at the price of {adr_px} on {row['date']}\n")
                    trade_records.append(f"We sold {stock_pos} shares of underlying stock at the price of {stock_px} USD ({stock_px_fx} foreign dollars) on {row['date']}\n")
                    stock_pos, adr_pos = 0, 0
                    prev_cash, prev_adr_pos = cash, adr_pos
                    if cash > portfolio_value_before_entering:
                        hits.append(1)
                    else:
                        hits.append(0)
                    
            # The 2nd trade of condition 2 falls on the next day
            if not (enter_cond1 or exit_cond1) and merged_df.loc[index+1, "date"] <= end_date:
                # Check and possibly trade condition 2
                enter_cond2 = (diff_record_bid_cond2[-1] > ask_mean_cond2 + entry_cond2_val*bid_std_cond2 
                               and diff_record_bid_cond2[-1] <= ask_mean_cond2 + stop_loss_cond2*bid_std_cond2
                               and stock_pos == 0 and adr_pos == 0)
                exit_cond2 = ((diff_record_ask_cond2[-1] < ask_mean_cond2 + exit_cond2_val*ask_std_cond2 
                              or diff_record_bid_cond2[-1] > ask_mean_cond2 + stop_loss_cond2*bid_std_cond2)
                              and stock_pos > 0 and adr_pos < 0)
                    
                if enter_cond2:
                    portfolio_value_before_entering = cash
                    units = min(int((0.5*cash)/row['adr_close_per_unit']),int((0.5*cash)/(row['stock_close_per_unit']/merged_df.loc[index+1,'avg_bid_non_us_before'])))
                    adr_quantity = int(units*row["adr_num_per_unit"])
                    stock_quantity = int(units*row["stock_num_per_unit"])
                    stock_pos += stock_quantity
                    stock_px_fx = merged_df.loc[index+1,'stock_open']*long_multiplier
                    stock_px = stock_px_fx/merged_df.loc[index+1,'avg_bid_non_us_at']
                    cash -= stock_px*stock_quantity
                    prev_cash, prev_adr_pos = cash, adr_pos
                    
                    adr_pos -= adr_quantity
                    adr_px = merged_df.loc[index+1,'adr_open']*short_multiplier
                    cash += adr_quantity*adr_px
                
                    trade_records.append("Opening positions:\n")
                    # Times in EST
                    trade_records.append(f"We bought {stock_quantity} shares of underlying stock at the price of {stock_px} USD ({stock_px_fx} foreign dollars) on {row['date']}\n")
                    trade_records.append(f"We sold {adr_quantity} shares of ADR at the price of {adr_px} on {merged_df.loc[index+1,'date']}\n")
                    
                elif exit_cond2:
                    stock_px_fx = merged_df.loc[index+1,'stock_open']
                    stock_px = stock_px_fx/merged_df.loc[index+1,'avg_ask_non_us_at']*short_multiplier
                    cash += stock_pos*stock_px
                    prev_cash, prev_adr_pos = cash, adr_pos
                    
                    adr_px = merged_df.loc[index+1,'adr_open']*long_multiplier
                    cash -= abs(adr_pos)*adr_px
                    trade_records.append("Closing positions:\n")
                    trade_records.append(f"We sold {stock_pos} shares of underlying stock at the price of {stock_px} USD ({stock_px_fx} foreign dollars) on {row['date']}\n")
                    trade_records.append(f"We bought {-adr_pos} shares of ADR at the price of {adr_px} on {merged_df.loc[index+1,'date']}\n")
                    stock_pos, adr_pos = 0, 0
                    if cash > portfolio_value_before_entering:
                        hits.append(1)
                    else:
                        hits.append(0)

    ret = (portfolio_values[-1] - starting_cash)/starting_cash

    return ret, trade_records, portfolio_values, hits, dates

In [ ]:
last_dates, last_portfolio_values = report_and_store_statistics(pairs_trade_v3a, 'logs3a.txt')

In [ ]:
plot_returns(last_dates, last_portfolio_values)

In [ ]:
"""
Variant 3b
- Begin each trade on either US market open or Asian market open
- Regressions are done for all data collected in lookback window

For each row:
    stock_open, stock_close, (assess condition 1), adr_open, adr_close, (assess condition 2)
    If not condition 2 - No action taken: 
        After first 2 events, Assess condition 1 (right before the US market opens ~ 9.29AM EST)
        If condition 1:
            Place trade on current and next row (First trade ADR on US market open, then trade stock on Asian market open)
    If not condition 1 - No action taken:
        After next 2 events occur, assess condition 2
        If condition 2:
            Place trade on next row (First trade ADR on Asian market open, then trade stock on US market open)
    
start_date: First date (EST) we may place a trade
end_date: Last date (EST) we may place a trade
portfolio_values: Stores value of portfolio at each date from one day before the start_date, to the end_date, when the Asian market opens
"""
def pairs_trade_v3b(merged_df, lookback = 100, cash = 100000, entry_cond1_val = 1, entry_cond2_val = 1, 
                    exit_cond1_val = 0, exit_cond2_val = 0, stop_loss_cond1 = 3, stop_loss_cond2 = 3, 
                    start_date = "2016-01-01", end_date = "2021-01-31", bps = 20):
    
    short_multiplier = 1 - 0.0001*bps
    long_multiplier = 1 + 0.0001*bps
    # We assume lookback is given in terms of days
    lookback *= 2
    starting_cash = cash
    stock_pos, adr_pos = 0, 0
    prev_cash, prev_adr_pos = cash, adr_pos
    diff_record_bid = deque(maxlen = lookback)
    diff_record_ask = deque(maxlen = lookback)
    trade_records = []
    portfolio_values = []
    dates = []
    hits = []
    enter_cond1, exit_cond1, enter_cond2, exit_cond2 = False, False, False, False

    for index, row in merged_df.iterrows():
                    
        # check if there is a px diff between close and stock_close effective
        # If len(diff_record) < lookback, we do not place any trade
        if index+1 < len(merged_df) and index > 0:
            
            # Add portfolio value for the day before
            prev_date = merged_df.loc[index - 1, "date"]
            if row["date"] >= start_date and prev_date <= end_date:
                dates.append(prev_date)
                portfolio_values.append(prev_cash + prev_adr_pos*merged_df.loc[index - 1, 'adr_close'] + stock_pos*(row['stock_open']/row['avg_ask_non_us_at']))
            
            # Before US Market Opens
            diff_record_bid.append(merged_df.loc[index-1,'adr_close_per_unit'] - row['stock_close_per_unit']/row['avg_bid_us_before'])
            diff_record_ask.append(merged_df.loc[index-1,'adr_close_per_unit'] - row['stock_close_per_unit']/row['avg_ask_us_before'])
            if len(diff_record_bid) == lookback and row["date"] >= start_date and row["date"] <= end_date:
                prev_cash, prev_adr_pos = cash, adr_pos
                ask_mean = np.array(diff_record_ask).mean()
                bid_std = np.array(diff_record_bid).std()
                ask_std = np.array(diff_record_ask).std()
                if not (enter_cond2 or exit_cond2):
                    # Check and possibly trade condition 1
                    enter_cond1 = (diff_record_bid[-1] > ask_mean + entry_cond1_val*bid_std
                                   and diff_record_bid[-1] <= ask_mean + stop_loss_cond1*bid_std
                                   and stock_pos == 0 and adr_pos == 0)
                    exit_cond1 = ((diff_record_ask[-1] < ask_mean + exit_cond1_val*ask_std
                                  or diff_record_bid[-1] > ask_mean + stop_loss_cond1*bid_std)
                                  and stock_pos > 0 and adr_pos < 0)
                    
                    if enter_cond1:
                        portfolio_value_before_entering = cash
                        units = min(int((0.5*cash)/merged_df.loc[index-1,'adr_close_per_unit']),int((0.5*cash)/(row['stock_close_per_unit']/row['avg_bid_us_before'])))
                        adr_quantity = int(units*row["adr_num_per_unit"])
                        stock_quantity = int(units*row["stock_num_per_unit"])
                        adr_pos -= adr_quantity
                        adr_px = row['adr_open']*short_multiplier
                        cash += adr_quantity*adr_px

                        stock_pos += stock_quantity
                        stock_px_fx = merged_df.loc[index+1,'stock_open']*long_multiplier
                        stock_px = stock_px_fx/merged_df.loc[index+1,'avg_bid_non_us_at']
                        cash -= stock_px*stock_quantity
                        prev_cash, prev_adr_pos = cash, adr_pos
                        trade_records.append("Opening positions:\n")
                        # Times in EST
                        trade_records.append(f"We sold {adr_quantity} shares of ADR at the price of {adr_px} on {row['date']}\n")
                        trade_records.append(f"We bought {stock_quantity} shares of underlying stock at the price of {stock_px} USD ({stock_px_fx} foreign dollars) on {row['date']}\n")

                    elif exit_cond1:

                        adr_px = row['adr_open']*long_multiplier
                        cash -= abs(adr_pos)*adr_px
                        stock_px_fx = merged_df.loc[index+1,'stock_open']*short_multiplier
                        stock_px = stock_px_fx/merged_df.loc[index+1,'avg_ask_non_us_at']
                        cash += stock_pos*stock_px
                        trade_records.append("Closing positions:\n")
                        trade_records.append(f"We bought {-adr_pos} shares of ADR at the price of {adr_px} on {row['date']}\n")
                        trade_records.append(f"We sold {stock_pos} shares of underlying stock at the price of {stock_px} USD ({stock_px_fx} foreign dollars) on {row['date']}\n")
                        stock_pos, adr_pos = 0, 0
                        prev_cash, prev_adr_pos = cash, adr_pos
                        if cash > portfolio_value_before_entering:
                            hits.append(1)
                        else:
                            hits.append(0)
                        
            # Before Asian Market Opens
            diff_record_bid.append(row['adr_close_per_unit'] - row['stock_close_per_unit']/merged_df.loc[index+1,'avg_bid_non_us_before'])
            diff_record_ask.append(row['adr_close_per_unit'] - row['stock_close_per_unit']/merged_df.loc[index+1,'avg_ask_non_us_before'])
            # The 2nd trade of condition 2 falls on the next day
            if len(diff_record_bid) == lookback and row["date"] >= start_date and merged_df.loc[index+1,"date"] <= end_date:
                ask_mean = np.array(diff_record_ask).mean()
                bid_std = np.array(diff_record_bid).std()
                ask_std = np.array(diff_record_ask).std()
                if not (enter_cond1 or exit_cond1):
                    # Check and possibly trade condition 2
                    enter_cond2 = (diff_record_bid[-1] > ask_mean + entry_cond2_val*bid_std
                                   and diff_record_bid[-1] <= ask_mean + stop_loss_cond2*bid_std
                                   and stock_pos == 0 and adr_pos == 0)
                    exit_cond2 = ((diff_record_ask[-1] < ask_mean + exit_cond2_val*ask_std
                                  or diff_record_bid[-1] > ask_mean + stop_loss_cond2*bid_std)
                                  and stock_pos > 0 and adr_pos < 0)

                    if enter_cond2:
                        portfolio_value_before_entering = cash
                        units = min(int((0.5*cash)/row['adr_close_per_unit']),int((0.5*cash)/(row['stock_close_per_unit']/merged_df.loc[index+1,'avg_bid_non_us_before'])))
                        adr_quantity = int(units*row["adr_num_per_unit"])
                        stock_quantity = int(units*row["stock_num_per_unit"])
                        stock_pos += stock_quantity
                        stock_px_fx = merged_df.loc[index+1,'stock_open']*long_multiplier
                        stock_px = stock_px_fx/merged_df.loc[index+1,'avg_bid_non_us_at']
                        cash -= stock_px*stock_quantity
                        prev_cash, prev_adr_pos = cash, adr_pos
                        
                        adr_pos -= adr_quantity
                        adr_px = merged_df.loc[index+1,'adr_open']*short_multiplier
                        cash += adr_quantity*adr_px
                        trade_records.append("Opening positions:\n")
                        # Times in EST
                        trade_records.append(f"We bought {stock_quantity} shares of underlying stock at the price of {stock_px} USD ({stock_px_fx} foreign dollars) on {row['date']}\n")
                        trade_records.append(f"We sold {adr_quantity} shares of ADR at the price of {adr_px} on {merged_df.loc[index+1,'date']}\n")

                    elif exit_cond2:
                        stock_px_fx = merged_df.loc[index+1,'stock_open']
                        stock_px = stock_px_fx/merged_df.loc[index+1,'avg_ask_non_us_at']*short_multiplier
                        cash += stock_pos*stock_px
                        prev_cash, prev_adr_pos = cash, adr_pos
                        
                        adr_px = merged_df.loc[index+1,'adr_open']*long_multiplier
                        cash -= abs(adr_pos)*adr_px
                        trade_records.append("Closing positions:\n")
                        trade_records.append(f"We sold {stock_pos} shares of underlying stock at the price of {stock_px} USD ({stock_px_fx} foreign dollars) on {row['date']}\n")
                        trade_records.append(f"We bought {-adr_pos} shares of ADR at the price of {adr_px} on {merged_df.loc[index+1,'date']}\n")
                        stock_pos, adr_pos = 0, 0
                        if cash > portfolio_value_before_entering:
                            hits.append(1)
                        else:
                            hits.append(0)

    ret = (portfolio_values[-1] - starting_cash)/starting_cash

    return ret, trade_records, portfolio_values, hits, dates

In [ ]:
last_dates, last_portfolio_values = report_and_store_statistics(pairs_trade_v3b, 'logs3b.txt')

In [ ]:
plot_returns(last_dates, last_portfolio_values)